# Extract sub-question from query

In [1]:
from query_decomposer import QueryDecomposer
qd = QueryDecomposer()

In [23]:
chat_history, sub_question = qd.decompose_question_step("Who was president of America the year Citibank was founded?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [25]:
chat_history

[{'generated_text': [{'role': 'system',
    'content': "\n    ## Question Decomposition Specialist:\n    You are an expert at breaking down difficult problems into simple problems by analysing them.\n    The user needs you to answer a complex question for them which is hard to answer directly.\n    Answer the question by decomposing it and tell the user at the right time when the problem can be solved.\n\n    ## Constraints:\n    Forget all knowledge you've learned before and decompose the user's question based only on the user's answers.\n    To make it easier for the user to answer, only ask one simple question at a time.\n    You can only decompose the question, do not answer it directly.\n    Only write one sentence for your answer containing a simple question.\n\n    ## Examples:\n    User: What is the award that the director of film Wearing Velvet Slippers Under A Golden Umbrella won?\n    You: Who is the director of Wearing Velvet Slippers Under A Golden Umbrella?\n    User: The

In [5]:
sub_question

'What year was Citibank founded?'

# Retrieve context for sub-question

In [4]:
from retrieval import Retrieval
retriever = Retrieval("..\context\knowledge_base")

c:\Users\Alex\ProjectCSurvival\rag\retrieval.py:153: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(summary_files) == len(summary_embeddings), "Summary text files should directly map to summary embedding files.")
c:\Users\Alex\ProjectCSurvival\rag\retrieval.py:188: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(embedding_files) == len(embedding_texts), "Embedding raw text files should directly map to embedding data files.")


In [6]:
context, article = retriever.get_context(sub_question, num_contexts=1, use_sparse_retrieval=True)
print(context)

Embedding user query for dense retrieval:


c:\Users\Alex\anaconda3\envs\gpu_tf\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Finding best article to use as context with sparse retrieval:
Using Wikipedia article: Apollo for context
Found 232 chunks for within context.
Finding best article chunks to use as context with dense retrieval:
Context successfully retrieved.
In spaceflight, the 1960s and 1970s NASA program for orbiting and landing astronauts on the Moon was named after Apollo, by NASA manager Abe Silverstein:


# Retrieve sub-answer from context

Use the correct context to retrieve the correct sub-answer.

In [ ]:
context = "Citibank, N.A. () is the primary U.S. banking subsidiary of Citigroup, a financial services multinational corporation. Citibank was founded in 1812 as City Bank of New York, and later became First National City Bank of New York. The bank has branches in 19 countries. The U.S. branches are concentrated in six metropolitan areas, New York City, Chicago, Los Angeles, San Francisco, Washington, D.C., and Miami."

In [12]:
sub_answer = qd.answer_question_using_context(
    sub_question,
    context)
sub_answer

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Citibank was founded in 1812.'

In [20]:
from retrieval import DenseRetrieval

uncertainty = DenseRetrieval.compare_documents(
    retriever.embedding_model.get_embedding(sub_answer),
    retriever.embedding_model.get_embedding("I don't know.")
)

if uncertainty > 0.9:
    print("Model does not know answer.")
    pass # Get new evidence
else:
    print("Model knows answer.")
    pass # Ask new sub-question.

Model knows answer.


In [52]:
chat_history = [{'generated_text': [{'role': 'system',
    'content': "\n    ## Question Decomposition Specialist:\n    You are an expert at breaking down difficult problems into simple problems by analysing them.\n    The user needs you to answer a complex question for them which is hard to answer directly.\n    Answer the question by decomposing it and tell the user at the right time when the problem can be solved.\n\n    ## Constraints:\n    Forget all knowledge you've learned before and decompose the user's question based only on the user's answers.\n    To make it easier for the user to answer, only ask one simple question at a time.\n    You can only decompose the question, do not answer it directly.\n    Only write one sentence for your answer containing a simple question.\n\n    ## Examples:\n    User: What is the award that the director of film Wearing Velvet Slippers Under A Golden Umbrella won?\n    You: Who is the director of Wearing Velvet Slippers Under A Golden Umbrella?\n    User: The director of Wearing Velvet Slippers Under A Golden Umbrella is Maung Wunna.\n    You: What awards has Maung Wunna won?\n    User: Maung Wunna won the Myanmar Motion Picture Academy Awards.\n    You: That's enough.\n    \n    User: Which film's director died earlier, Condemned Women or Faces In The Dark?\n    You: What is the director of the film Condemned Women?\n    User: The director of the film Condemned Women is Lew Landers.\n    You: What is the director of the film Faces In The Dark?\n    User: The director of the film Faces In The Dark is David Eady.\n    You: When did Lew Landers die?\n    User: Lew Landers died on 16 December 1962.\n    You: When did David Eady die?\n    User: David Eady died on April 5, 2009.\n    You: That's enough.\n    "},
   {'role': 'user',
    'content': "Let's break down this complex question: Who was president of America the year Citibank was founded?"},
   {'role': 'assistant', 'content': 'What year was Citibank founded?'}]}]

In [53]:
new_chat_history = chat_history[0]['generated_text'].copy()

new_chat_history.append(
    {'role': 'user', 'content': sub_answer}
)

new_chat_history

[{'role': 'system',
  'content': "\n    ## Question Decomposition Specialist:\n    You are an expert at breaking down difficult problems into simple problems by analysing them.\n    The user needs you to answer a complex question for them which is hard to answer directly.\n    Answer the question by decomposing it and tell the user at the right time when the problem can be solved.\n\n    ## Constraints:\n    Forget all knowledge you've learned before and decompose the user's question based only on the user's answers.\n    To make it easier for the user to answer, only ask one simple question at a time.\n    You can only decompose the question, do not answer it directly.\n    Only write one sentence for your answer containing a simple question.\n\n    ## Examples:\n    User: What is the award that the director of film Wearing Velvet Slippers Under A Golden Umbrella won?\n    You: Who is the director of Wearing Velvet Slippers Under A Golden Umbrella?\n    User: The director of Wearing V

In [54]:
new_chat_history[-1]

{'role': 'user', 'content': 'Citibank was founded in 1812.'}

In [55]:
chat_history, sub_question = qd.decompose_question_step(new_chat_history)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [57]:
chat_history

[{'generated_text': [{'role': 'system',
    'content': "\n    ## Question Decomposition Specialist:\n    You are an expert at breaking down difficult problems into simple problems by analysing them.\n    The user needs you to answer a complex question for them which is hard to answer directly.\n    Answer the question by decomposing it and tell the user at the right time when the problem can be solved.\n\n    ## Constraints:\n    Forget all knowledge you've learned before and decompose the user's question based only on the user's answers.\n    To make it easier for the user to answer, only ask one simple question at a time.\n    You can only decompose the question, do not answer it directly.\n    Only write one sentence for your answer containing a simple question.\n\n    ## Examples:\n    User: What is the award that the director of film Wearing Velvet Slippers Under A Golden Umbrella won?\n    You: Who is the director of Wearing Velvet Slippers Under A Golden Umbrella?\n    User: The

In [16]:
from retrieval import DenseRetrieval
DenseRetrieval.compare_documents(
    retriever.embedding_model.get_embedding(sub_answer),
    retriever.embedding_model.get_embedding("I don't know.")
)

tensor([0.6709], dtype=torch.float16)

In [ ]:
"The Andorran opening to the capitalist economy resulted in two axes: mass tourism and the country's tax exemption. The first steps toward the capitalist boom date from the 1930s, with the construction of FHASA and the creation of professional banking with Banc Agrícol (1930) and Crèdit Andorrà (1949), later with Banca Mora (1952), Banca Cassany (1958) and SOBANCA (1960). Shortly after, activities such as skiing and shopping become a tourist attraction, with the inauguration of ski resorts and cultural entities in the late 1930s. All in all, a renovated hotel industry has developed. In April 1968 a social health insurance system was created (CASS)."